In [12]:
import os
import cv2
import json
import imutils
from ocr.config import output_path, OUTPUT_ROOT
import random
import shutil
import csv
from config import  *
import pandas as pd
import difflib
import math
import numpy as np

# Crop pill images

In [2]:
src = "data/public_train/pill/image/" #jpg
src_label = "data/public_train/pill/label/"

for file in os.listdir(src):
    if "jpg" in file:
        img = cv2.imread(src + file)
        json_f = open(src_label + file.replace("jpg", "json"), mode='r')
        lab = json.load(json_f)

        img_show = imutils.resize(img, width=960)

        for i, box in enumerate(lab):
            pill = img[box['y']:box['y']+box['h'], box['x']:box['x']+box['w']]
            cls = box['label']

            crop_dir = output_path('{}/'.format(f'croped/public_train/{cls}'))
            crop_output_path = os.path.join(crop_dir, file.replace('.', f'_{i}.'))
            cv2.imwrite(crop_output_path, pill)

#             cv2.imshow('pill', pill)
#             cv2.imshow('image', img_show)

#             k = cv2.waitKey(0)
#             if k == ord('q'):
#                 break
#         if k == ord('q'):
#                 break

# cv2.destroyAllWindows()

# Make sample class

In [ ]:
src = output_path('croped/public_train/')
dst = output_path('pill_sample_class/')

for folder in os.listdir(src):
    files = os.listdir(src + folder)
    file = random.choice(files)
    print(file)
    shutil.copy(src + folder + '/' + file, dst + folder + "_" + file)

# **Draw box detect yolo (public train)**

In [5]:
detected_file = OUTPUT_ROOT + "csv_submit/result_final_filtered.csv"
out_fnam = OUTPUT_ROOT + "csv_submit/result_final_filtered_remove_blank.csv"

in_fnam = detected_file

with open(in_fnam, newline='') as in_file:
    with open(out_fnam, 'w', newline='') as out_file:
        writer = csv.writer(out_file)
        for row in csv.reader(in_file):
            if row:
                writer.writerow(row)



In [35]:
detected_file = OUTPUT_ROOT + "csv_submit/result_final_filtered_remove_blank.csv"
# INPUT_ROOT = 'c:/Users/Admin/Desktop/Course/AIO2022/TeamHungAn/Medicine_Pill_Image_Recognition_Challenge_pineline'
input_dir = "data/public_test/pill/image/" #INPUT_ROOT
output_dir = OUTPUT_ROOT + "public_test/pill/image/"
image_name_pre = None

df = pd.read_csv(detected_file)
img = None
for i, row in df.iterrows():
    if row['image_name'] != image_name_pre:
        # save output before load new image
        if image_name_pre != None:
            cv2.imwrite(output_dir + image_name_pre, img)

        # load input image moi
        image_name_pre = row['image_name']
        img = cv2.imread(input_dir + image_name_pre)
    img = cv2.rectangle(img, (row['x_min'], row['y_min']), (row['x_max'], row['y_max']), (0, 255, 255), 4)
    img_show = imutils.resize(img, width=640)
#     cv2.imshow("pills", img_show)

#     if cv2.waitKey(0) == ord('q'):
#         break
# cv2.destroyAllWindows()

## process public_train

In [16]:
src = "data/public_train/ocr/label/"
output = "output/public_train/ocr/public_train_pres_output.csv"

def get_all_drugnames(mapping_file):
    df = pd.read_csv(mapping_file)
    DrugList = df['Drug name'].tolist()
    # LabelCells = df['label'].tolist()
    # print(LabelCells)
    AllDrugNames = []
    for cell in DrugList:
        try:
            names = cell.split(" || ")
            for name in names:
                name = name.replace('"', '')
                # print("'"+name+"'")
                AllDrugNames.append(name)
        except:
            if math.isnan(cell):
                print(cell)
    return DrugList, AllDrugNames

def write_output_csv(data, filepath, mode):
    presoutput = open(filepath, mode, encoding='UTF-8', newline='')
    writer = csv.writer(presoutput)
    writer.writerow(data)
    presoutput.close()

def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list

def get_all_disease_code(icd10_file):
    df = pd.read_csv(icd10_file, encoding='utf-8')
    diseaselist = df['Mã'].tolist()
    print("diseaselist first: ", len(diseaselist))
    diseaselist = unique(diseaselist)
    print("diseaselist unique: ", len(diseaselist))
    return diseaselist

def find_disease_code(diagnose_boxes, diseaselist):
    codes = []
    codestring = ""
    # print(diagnose_boxes)
    for box in diagnose_boxes:
        text = box
        if len(text) > 15:
            # print(text)
            for code in diseaselist:
                if code in text:
                    codes.append(code)
    l = sorted(codes, key=len)
    codes = [j for i, j in enumerate(l) if all(j not in k for k in l[i + 1:])]
    print("disease_codes: ", codes)
    for i, code in enumerate(codes):
        if i == len(codes)-1:
            codestring += f'{code}'
        else:
            codestring += f'{code}-'
    return codestring

icd10_file = BASE_DIR + "/Medicine_Pill_Image_Recognition_Challenge_pineline/ocr/ICD-10.1_8_2022.csv"
mapping_file = BASE_DIR + "/Medicine_Pill_Image_Recognition_Challenge_pineline/ocr/label-name.csv"
AllDiseaseCodes = get_all_disease_code(icd10_file)
DrugList, AllDrugNames = get_all_drugnames(mapping_file)

header = ['PresName', 'DiseaseCodes', 'DrugName', 'Labels']
write_output_csv(data=header, filepath=output, mode='w')
def get_num(elem):
    return int(elem.split(".")[0].split("_")[-1])
files = os.listdir(src)
files.sort(key=get_num)
for file in files:
    if "json" in file:
        json_f = open(src+file, mode='r', encoding='utf-8')
        pres_data = json.load(json_f)

        diagnose_boxes = []
        out = []

        for obj in pres_data:
            print(obj)
            text = obj["text"]
            if obj["label"] == "diagnose":
                diagnose_boxes.append(obj["text"])
            elif obj["label"] == "drugname":
                # detect drugname and ids
                drugnames_predict = difflib.get_close_matches(text, AllDrugNames)
                probs = []
                for name in drugnames_predict:
                    prob = difflib.SequenceMatcher(None, text, name).ratio()
                    probs.append(prob)
                if drugnames_predict != []:
                    drugname = drugnames_predict[np.argmax(probs)]
                mapping = obj["mapping"]
                out.append([file, drugname, mapping])

        # Find disease codes
        codestring = find_disease_code(diagnose_boxes, AllDiseaseCodes)
        for row in out:
            file, drugname, mapping = row
            write_output_csv(data=[file, codestring, drugname, mapping], filepath=output, mode='a')

        

diseaselist first:  36702
diseaselist unique:  13200
{'id': 1, 'text': '1/1', 'label': 'other', 'box': [58, 28, 113, 58]}
{'id': 3, 'text': 'LHANG', 'label': 'other', 'box': [153, 60, 208, 76]}
{'id': 4, 'text': 'Số phiếu 1646/2019', 'label': 'other', 'box': [372, 91, 535, 123]}
{'id': 5, 'text': '3648383', 'label': 'other', 'box': [626, 100, 688, 118]}
{'id': 6, 'text': 'TOA THUỐC BHYT', 'label': 'other', 'box': [260, 122, 508, 152]}
{'id': 9, 'text': 'Nam', 'label': 'other', 'box': [618, 160, 655, 177]}
{'id': 10, 'text': 'Nữ', 'label': 'other', 'box': [685, 160, 708, 177]}
{'id': 13, 'text': '35', 'label': 'other', 'box': [366, 207, 387, 224]}
{'id': 14, 'text': '208', 'label': 'other', 'box': [436, 207, 468, 225]}
{'id': 15, 'text': '07327', 'label': 'other', 'box': [509, 207, 560, 225]}
{'id': 16, 'text': '35', 'label': 'other', 'box': [300, 208, 322, 225]}
{'id': 17, 'text': 'MMHg Thân nhiệt:', 'label': 'other', 'box': [450, 231, 595, 255]}
{'id': 18, 'text': 'lần/phút Huyết áp:'

TypeError: write_output_csv() missing 2 required positional arguments: 'filepath' and 'mode'